# Notebook to compute the Degree Lighting Weeks index at certain depth 'dDLW' index 

In [1]:
##### Open .CSV file as pandas dataframe
import pandas as pd
import csv 

In [2]:
df_PARd = pd.read_csv('PAR[Kd].csv')

# Equation

𝑃𝐴𝑅𝑑 = 𝑃𝐴𝑅 ∗ 𝑒^((−𝐾𝑑_490∗𝑑𝑒𝑝𝑡ℎ))


In [ ]:
df_PARd.head()

In [5]:
def arrange_df (df):
    
    'This function reindex, sort_values, interpolate NaN´s and append the average of the week and average of month'

    df = df.reindex(columns=['site_name','year','month','week','days','PAR','Kd_490','depth2','depth3','depth5','depth_site','PARd','PARd2','PARd3','PARd5','PARsite'])
    # df = df.sort_values(by=['site_name','year','month','week','days'])
    # df = df.interpolate()

    # Here is used 'PARsite', but can be used any other metric; e.g. df['PARd2'] = ... 
    df['PARdsite'] = df.groupby(['site_name','year','month','week','days'])['PARsite'].transform('mean')
    df['PARdsite_week'] = df.groupby(['site_name','year','month','week'])['PARsite'].transform('mean')

    # The anomaly can be computed using different approaches; here is used the 'weekly climatology'
    
    df['PARdsite_Wclim'] = df.groupby(['site_name','week'])['PARsite'].transform('mean')
    df['PARdsite_anomaly'] = df['PARdsite'] - df['PARdsite_Wclim']
    return df

In [6]:
df_PARd = arrange_df(df_PARd)

In [7]:
df_PARd.head( )

,site_name,year,month,week,days,PAR,Kd_490,depth2,depth3,depth5,depth_text,PARd,PARd2,PARd3,PARd5,PARsite,PARdsite,PARdsite_week,PARdsite_Wclim,PARdsite_anomaly
0,CAN,1998,1,0,1,33.334000,0.078,2,3,5,3.0,30.832764,28.519210,26.379254,22.569014,26.379254,26.379254,26.052305,22.557205,3.822049
1,CAN,1998,1,0,2,32.917000,0.078,2,3,5,3.0,30.447054,28.162442,26.049257,22.286681,26.049257,26.049257,26.052305,22.557205,3.492052
2,CAN,1998,1,0,3,32.500000,0.079,2,3,5,3.0,30.031298,27.750118,25.642217,21.894601,25.642217,25.642217,26.052305,22.557205,3.085012
3,CAN,1998,1,0,4,33.129000,0.079,2,3,5,3.0,30.612518,28.287189,26.138492,22.318346,26.138492,26.138492,26.052305,22.557205,3.581287
4,CAN,1998,1,1,5,33.757999,0.080,2,3,5,3.0,31.162561,28.766670,26.554983,22.628664,26.554983,26.554983,26.783961,22.123984,4.430999


### To get the "week anomaly" the groupby function should be in week periods

In [18]:
PARdsiteW_anomaly = df_PARd.groupby(['site_name','year','month','week'])['PARdsite_week'].mean()-df_PARd.groupby(['site_name','year','month','week'])['PARdsite_Wclim'].mean()

In [19]:
PARdsiteW_anomaly.head()

site_name  year  month  week
CAN        1998  1      0       3.495100
                        1       4.659977
                        2       5.899450
                        3       7.046994
                        4       5.597439
dtype: float64

In [20]:
import numpy as np
PARdsiteW_anomaly = round(PARdsiteW_anomaly, 3)
hit = PARdsiteW_anomaly < 0
PARdsiteW_anomaly[hit] = 0

In [21]:
PARdsiteW_anomaly = pd.DataFrame(PARdsiteW_anomaly)

In [22]:
def dDLW (df):
    df = round(df.rolling(center=False,window=12).sum(),3) # Sum the values in the windows of "twelve" weeks
    df['site_name'] = df.index.get_level_values(level=0) # assign names and add new columns based on hiden levels
    df['year'] = df.index.get_level_values(level=1)
    df['month'] = df.index.get_level_values(level=2)
    df.columns = ['dDLW','site_name','year','month'] # Re-name the columns
    df = df.reset_index(level=(0,1,2,3), drop=True) # Drop the hiden levels
    df = df.reindex(columns = ['site_name','year','month','dDLW']) # Re-order the columns 
    return df

In [23]:
dDLW = dDLW(PARdsiteW_anomaly)

In [24]:
dDLW.to_csv('dDLWsite.csv')

,site_name,year,month,dDLW
0,CAN,1998,1,NaN
1,CAN,1998,1,NaN
2,CAN,1998,1,NaN
3,CAN,1998,1,NaN
4,CAN,1998,1,NaN
5,CAN,1998,2,NaN
6,CAN,1998,2,NaN
7,CAN,1998,2,NaN
8,CAN,1998,2,NaN
9,CAN,1998,2,NaN


In [25]:
# This is an alternative to export the data as a bit more 'neat'

table = pd.pivot_table(dDLW,index=["site_name","year","month"],values=["dDLW"])
table = table.reindex(columns=["dDLW"])  # reordering the index
#table.to_csv('DLW.csv')
table.to_csv('dDLWsite.csv')  # table with enterely dataset